In [ ]:

import seaborn as sns
import numpy as np
import torch
import torch.nn as nn
from torchvision.datasets import STL10, CIFAR10, CIFAR100, ImageFolder, ImageNet
import pandas as pd
import os

from datasets.imagenette import ImageNette
from hash_classifier import use_imagenet_categories, balance_classes

sns.set_style("ticks", {'axes.grid': True})

In [ ]:
VAL_DATASET = 'imagenet_val'
TRAIN_DATASET = 'imagenet_train'
NUM_CLASSES = 1000
BALANCE_CLASSES = True
USE_NORMALIZED_BIT_VALUES = False
SEEDS = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
MODEL_BASE_PATH_CATEGORY_MODELS = './pretrained_models/category_classifier_models'
MODEL_BASE_PATH_CLASS_MODELS = './pretrained_models/class_classifier_models'
TRAIN_HASH_DIR = f'dataset_hashes/{TRAIN_DATASET}/{TRAIN_DATASET}_original_with_targets.csv'
VAL_HASH_DIR = f'dataset_hashes/{VAL_DATASET}/{VAL_DATASET}_original_with_targets.csv'

In [ ]:
# get the dataset and the calcualted hashes from the csv files
def get_dataset(dataset_name: str, additional_transforms=None):
    if dataset_name.lower() == 'stl10':
        dataset = STL10(root='data', split='train', download=True)
    elif dataset_name.lower() == 'imagenette':
        dataset = ImageNette(root='data', train=True, download=True)
    elif dataset_name.lower() == 'cifar10':
        dataset = CIFAR10(root='data', train=True, download=True)
    elif dataset_name.lower() == 'cifar100':
        dataset = CIFAR100(root='data', train=True, download=True)
    elif dataset_name.lower() == 'imagenet_test':
        dataset = ImageFolder(root='data/ILSVRC2012_test')
    elif dataset_name.lower() == 'imagenet_train':
        dataset = ImageNet(root='data/ILSVRC2012', split='train')
    elif dataset_name.lower() == 'imagenet_val':
        dataset = ImageNet(root='data/ILSVRC2012', split='val')
    else:
        raise RuntimeError(f'Dataset with name {dataset_name} was not found.')
    return dataset


def get_prediction_vecs(model, dataloader):
    # get the predictions for the validation set
    predictions = []
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.cuda(), y.cuda()
            predictions.append(model(x).softmax(1).cpu())

    predictions = torch.cat(predictions)

    return predictions


def topk_accuracy(output, target, topk=(1,)):
    """
    Computes the accuracy over the k top predictions for the specified values of k.
    Taken from the PyTorch ImageNet example (https://github.com/pytorch/examples/blob/master/imagenet/main.py)
    and slightly modified.
    """
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.T
        correct = pred == target.view(1, -1).expand_as(pred)

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res


def get_acc_per_class(targets, predictions):
    # get the accuracy per class
    with torch.no_grad():
        accuracy_per_class = []
        for target in np.unique(targets):
            class_indices = (torch.tensor(targets) == target).nonzero().flatten()
            class_predictions = predictions[class_indices]
            top1 = topk_accuracy(class_predictions, torch.tensor(targets[class_indices]), topk=(1,))[0]

            accuracy_per_class.append(top1)

        accuracy_per_class = torch.cat(accuracy_per_class)
    return accuracy_per_class

# get the training data
train_df = pd.read_csv(TRAIN_HASH_DIR)
# get the validation data
val_df = pd.read_csv(VAL_HASH_DIR)
val_dataset = get_dataset(VAL_DATASET)

In [ ]:
val_targets = val_df['target'].to_numpy()
val_vecs = np.array([[int(bit) for bit in list(hash)] for hash in val_df['hash_bin'].to_numpy()])
if USE_NORMALIZED_BIT_VALUES:
    val_vecs = (val_vecs - 0.5) * 2
val_indices = np.concatenate([np.where(val_targets == i) for i in range(NUM_CLASSES)], axis=1).flatten()
val_set = torch.utils.data.TensorDataset(torch.tensor(val_vecs).float(), torch.tensor(val_targets))
val_loader = torch.utils.data.DataLoader(val_set, batch_size=256, num_workers=8, shuffle=False)

In [ ]:
# load the pretrained classifier
# for the vanilla hash classifier a dropout of 0.2 was used
neuron_nums = [96, 2048, 4096, 2048, NUM_CLASSES]
layers = []
for i in range(1, len(neuron_nums)):
    layers.extend([
        nn.Linear(neuron_nums[i - 1], neuron_nums[i]),
        nn.BatchNorm1d(neuron_nums[i]),
        nn.Dropout(p=0),
        nn.ReLU()
    ])

nn_classifier = nn.Sequential(
    *layers
)


In [ ]:
# get the predictions for the validation set of each model trained with a different seed
class_classifier_prediction_dict = {}
for seed in SEEDS:
    model_file_name = f'imagenet_train_hash_classifier_stratified{"_balanced" if BALANCE_CLASSES else ""}_seed{seed}.pt'
    model_path = os.path.join(MODEL_BASE_PATH_CLASS_MODELS, model_file_name)
    nn_classifier.load_state_dict(torch.load(model_path))
    nn_classifier = nn_classifier.cuda()
    nn_classifier.eval()
    class_classifier_prediction_dict[seed] = get_prediction_vecs(nn_classifier, val_loader)
    nn_classifier.cpu()


In [ ]:
class_classifier_accs =  {
    'top1': [],
    'top5': [],
    'top10': []
}
for seed in SEEDS:
    # calculate the top-1, top-5 and top-10 accuracy
    class_classifier_accs['top1'].append(topk_accuracy(class_classifier_prediction_dict[seed], torch.tensor(val_targets), topk=[1])[0].item())
    class_classifier_accs['top5'].append(topk_accuracy(class_classifier_prediction_dict[seed], torch.tensor(val_targets), topk=[5])[0].item())
    class_classifier_accs['top10'].append(topk_accuracy(class_classifier_prediction_dict[seed], torch.tensor(val_targets), topk=[10])[0].item())
    print(f'Seed {seed}: ({class_classifier_accs["top1"][-1]}, {class_classifier_accs["top5"][-1]}, {class_classifier_accs["top10"][-1]})')

for key in class_classifier_accs.keys():
    class_classifier_accs[key] = np.array(class_classifier_accs[key])

print(f'Top1-Mean-Accuracy: {class_classifier_accs["top1"].mean():5.4f}%')
print(f'Top5-Mean-Accuracy: {class_classifier_accs["top5"].mean():5.4f}%')
print(f'Top10-Mean-Accuracy: {class_classifier_accs["top10"].mean():5.4f}%')
print(f'Top1-Mean-Standard-Deviation: {class_classifier_accs["top1"].std():5.4f}%')
print(f'Top5-Mean-Standard-Deviation: {class_classifier_accs["top5"].std():5.4f}%')
print(f'Top10-Mean-Standard-Deviation: {class_classifier_accs["top10"].std():5.4f}%')



In [ ]:
accuracy_per_class = get_acc_per_class(val_targets, class_classifier_prediction_dict[SEEDS[0]])

# print the 5 classes with the best accuracy
top5_classes_acc, top5_classes_idx = torch.topk(accuracy_per_class, 10)

print('Classes with the best prediction:')
for acc, idx in zip(top5_classes_acc, top5_classes_idx):
    print(f'{val_dataset.classes[idx]}: {acc:5.4f}%')

# Group the Imagenet Classes in Categories

In [ ]:
categories = pd.read_csv('./imagenet_categories_modified.csv')

category_val_df = val_df.copy(deep=True)

use_imagenet_categories(val_dataset, train_df)
use_imagenet_categories(val_dataset, category_val_df, modify_dataset=True)

category_val_targets = category_val_df['target'].to_numpy()
category_val_vecs = np.array([[int(bit) for bit in list(hash)] for hash in category_val_df['hash_bin'].to_numpy()])
if USE_NORMALIZED_BIT_VALUES:
    category_val_vecs = (category_val_vecs - 0.5) * 2

def get_category_data_indices(train_df, val_df, targets):
    if BALANCE_CLASSES:
        train_indices = balance_classes(train_df)
        val_indices = balance_classes(val_df)
    else:
        val_indices = [np.where(targets == i)[0] for i in range(NUM_CLASSES)]

    val_indices = np.concatenate(val_indices).flatten()

    return val_indices


In [ ]:
# load the pretrained category classifier
# for the category hash classifier a dropout of 0.3 was used
neuron_nums = [96, 2048, 4096, 2048, 85]
layers = []
for i in range(1, len(neuron_nums)):
    layers.extend([
        nn.Linear(neuron_nums[i - 1], neuron_nums[i]),
        nn.BatchNorm1d(neuron_nums[i]),
        nn.Dropout(p=0),
        nn.ReLU()
    ])

nn_category_classifier = nn.Sequential(
    *layers
)


In [ ]:
# get the predictions for the validation set of each model trained with a different seed
category_classifier_prediction_dict = {}
for seed in SEEDS:
    np.random.seed(seed)
    torch.manual_seed(seed)
    val_indices = get_category_data_indices(train_df, category_val_df, category_val_targets)
    current_category_val_set = torch.utils.data.TensorDataset(torch.tensor(category_val_vecs[val_indices]).float(), torch.tensor(category_val_targets[val_indices]))
    current_category_val_loader = torch.utils.data.DataLoader(current_category_val_set, batch_size=256, num_workers=8, shuffle=False)

    model_file_name = f'imagenet_train_categories_hash_classifier_stratified{"_balanced" if BALANCE_CLASSES else ""}_seed{seed}.pt'
    model_path = os.path.join(MODEL_BASE_PATH_CATEGORY_MODELS, model_file_name)
    nn_category_classifier.load_state_dict(torch.load(model_path))
    nn_category_classifier = nn_category_classifier.cuda()
    nn_category_classifier.eval()
    category_classifier_prediction_dict[seed] = get_prediction_vecs(nn_category_classifier, current_category_val_loader)
    nn_category_classifier.cpu()

In [ ]:
category_classifier_accs =  {
    'top1': [],
    'top5': [],
    'top10': []
}
for seed in SEEDS:
    # calculate the top-1, top-5 and top-10 accuracy
    np.random.seed(seed)
    torch.manual_seed(seed)
    val_indices = get_category_data_indices(train_df, category_val_df, category_val_targets)

    category_classifier_accs['top1'].append(topk_accuracy(category_classifier_prediction_dict[seed], torch.tensor(category_val_targets[val_indices]), topk=[1])[0].item())
    category_classifier_accs['top5'].append(topk_accuracy(category_classifier_prediction_dict[seed], torch.tensor(category_val_targets[val_indices]), topk=[5])[0].item())
    category_classifier_accs['top10'].append(topk_accuracy(category_classifier_prediction_dict[seed], torch.tensor(category_val_targets[val_indices]), topk=[10])[0].item())
    print(f'Seed {seed}: ({category_classifier_accs["top1"][-1]}, {category_classifier_accs["top5"][-1]}, {category_classifier_accs["top10"][-1]})')

for key in category_classifier_accs.keys():
    category_classifier_accs[key] = np.array(category_classifier_accs[key])

print(f'Top1-Mean-Accuracy: {category_classifier_accs["top1"].mean():5.4f}%')
print(f'Top5-Mean-Accuracy: {category_classifier_accs["top5"].mean():5.4f}%')
print(f'Top10-Mean-Accuracy: {category_classifier_accs["top10"].mean():5.4f}%')
print(f'Top1-Mean-Standard-Deviation: {category_classifier_accs["top1"].std():5.4f}%')
print(f'Top5-Mean-Standard-Deviation: {category_classifier_accs["top5"].std():5.4f}%')
print(f'Top10-Mean-Standard-Deviation: {category_classifier_accs["top10"].std():5.4f}%')


In [ ]:
seed = SEEDS[0]
np.random.seed(seed)
torch.manual_seed(seed)
val_indices = get_category_data_indices(train_df, category_val_df, category_val_targets)
accuracy_per_category = get_acc_per_class(category_val_targets[val_indices], category_classifier_prediction_dict[seed])

# print the 5 classes with the best accuracy
top5_category_acc, top5_category_idx = torch.topk(accuracy_per_category, 10)

print('Categories with the best prediction:')
for acc, idx in zip(top5_category_acc, top5_category_idx):
    print(f'{val_dataset.classes[idx]}: {acc:5.4f}%')
